# 01 Webscraping

### TODO (Nice to have)

- input = cphbusiness.dk
- lav input til en string istedet for liste
- gem scrape med side navnet + pickle dump til side navn
- QOL kun lav sleep når man henter mere end 5 pages.
- QQL omskrive metode navne
- skriv en funktion som renser review data/replace & tilføj lowercase clean fra datacleaning notebook
- Stor QQL lav en celle til sidst som kører det hele.

In [1]:
import bs4
import csv
import requests
import re
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
import random
import pickle
import time

workingProxyList=[] # used for proxies (free ones are too slow)
allShopsRequest = []
totalpageprShop=[] # used in multiThreadGetTotalPages
allPagesRequest = [] # used in multiThreadDownloader
reviewErrorCount = [] # used in multithreadwritecsvfile

In [3]:
#Kør kun denne celle hvis du vil bruge proxy

#with open('ipList2', 'rb') as file:
#    workingProxyList = pickle.load(file)

0


In [ ]:
allPagesRequest.append("https://dk.trustpilot.com/review/www.cphbusiness.dk")

print("OK")

In [2]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

                      
    return re.sub(emoj, '', data)

print("OK")

OK


In [ ]:
def multiThreadGetTotalPages(shop):
    loadingbar.update(1)
    listofpages=""
    totalpages=None
    req = requests.get(shop, timeout=10)
    soup = bs4.BeautifulSoup(req.content, 'html.parser')
    
    try:      
        totalpages = soup.find('a', attrs={'name':'pagination-button-last'}).text
        shopAndPage = {'shop': shop,'pages': totalpages}
        totalpageprShop.append(shopAndPage)
    except:       
        listofpages = [7,6,5,4,3,2,1]
        try:
            for content in listofpages:
                totalpages = soup.find('a', attrs={'name':'pagination-button-'+str(content)}).text
                if totalpages is not None:
                    print('Amount of review pages found:', totalpages)
                    shopAndPage = {'shop': shop,'pages': totalpages}
                    totalpageprShop.append(shopAndPage)
                    break
                
        except Exception as e:
            print('Error', e)
 
        
with tqdm(total=len(allShopsRequest)) as loadingbar: 
    with ThreadPoolExecutor(2) as ex:
        ex.map(multiThreadGetTotalPages, allShopsRequest)

In [ ]:
def multiThreadDownloader(shop,page):
    loadingbar.update(1)
    
    url = shop+'?page='+str(page)
    
    #Overvej at droppe proxy.. Servers er langsomme, timer ofte ud eller virker slet ikke. 
    #Selvom de er testet 5min forinden.
    if len(workingProxyList) > 0:
        for ip in workingProxyList:
            try:
                randomIpNumber = random.randint(0,len(workingProxyList))
                allPagesRequest.append(requests.get(url, timeout=15, proxies=workingProxyList[randomIpNumber]))
                print(len(allPagesRequest))
                break
            except:
                pass
    else:
        allPagesRequest.append(requests.get(url, timeout=10))
        
    time.sleep(10)


allpages=0    
for x in totalpageprShop:
    allpages = allpages+int(x['pages'])


with tqdm(total=allpages) as loadingbar:
    print(len(workingProxyList))
    for i in totalpageprShop:
        digitlist = []
        res = int(i['pages'])
        for digit in range(res):
            digitlist.append(digit)
        with ThreadPoolExecutor(4) as ex:
            ex.map(multiThreadDownloader,[i['shop']] * len(digitlist), digitlist)
    pickle.dump(allPagesRequest,open('dump','wb'))

In [ ]:
def multithreadwritecsvfile(page):
    loadingbar.update(1)
    with open('test.csv', 'a', newline='', encoding='utf-8') as output_file:
        output_writer = csv.writer(output_file)
        soup = bs4.BeautifulSoup(page.content,'html.parser')
        for content in soup.find_all('section', attrs={'class':'styles_reviewContentwrapper__zH_9M'}):
            rating=None
            splitted=None
            title=None
            review=None
            
            try: 
                rating = content.find('img',alt = True)
                splitted = rating.get('alt').split()
            except Exception as e:
                print('No rating found. Skipping...', e)
            
            try: 
                title = content.find('a', attrs={'name':'review-title'}).text
                title = remove_emojis(title)
            except Exception as e:
                print('No title found. Skipping...', e)
                
            try: 
                review = content.find('p', attrs={'class':'typography_typography__QgicV typography_body__9UBeQ typography_color-black__5LYEn typography_weight-regular__TWEnf typography_fontstyle-normal__kHyN3'}).text
                if len(review) > 2:
                    review = remove_emojis(review)
                    review = review.replace(",", " ")
                    review = review.replace(' "" ', "")
                    review = review.replace('  ', "")
                    review = review.replace(' \n ', "")
                    output_writer.writerow(['__label__'+splitted[2]+" ", review])
            except Exception as e:
                #print('No review found. Skipping...', e)
                reviewErrorCount.append(e);
            
with open('dump','rb') as file:
    allPagesRequest = pickle.load(file)
    
with tqdm(total=len(allPagesRequest)) as loadingbar: 
    with ThreadPoolExecutor(8) as ex:
        with open('test.csv', 'w', newline='', encoding='utf-8') as output_file:
            output_writer = csv.writer(output_file)
            output_writer.writerow(['rating','review'])
        ex.map(multithreadwritecsvfile, allPagesRequest)

print('Done. Skipped reviews:', len(reviewErrorCount))